# **1. 라이브러리 임포트**

In [1]:
import requests
import pprint
import json
import pandas as pd
import time

# **2. API 요청변수**

In [2]:
# api_public_key =  개인 API KEY 입력해야 오류나지 않음                              
# api_hash = 식별값 
limit = 100                                               # 한 번에 가져올 수 있는 최대 데이터 수
offset = 0                                                # 호출의 요청된 오프셋(건너뛴 결과 수)

# **3. 함수 만들기**
- 마블 캐릭터의 유니크한 id값이 URL 끝에 위치
- '/' 를 기준으로 맨 끝의 값을 가져오는 함수 

In [4]:
def replace_resourceURI_id(uri : str) :  
    '''
    매개 변수값을 "/" 기준으로 슬라이싱 하여 맨 끝의 값을 리턴함.

    Args:
        uri => 'http://gateway.marvel.com/v1/public/comics/24571'

    Returns: 
        uri.split("/") => ['http:', 'gateway', .... , '24571']
        uri.split("/")[-1] => '24571'

        최종 값은 '24571'
    '''
    return uri.split("/")[-1]

# **4. 데이터 수집**
- 기존에 수집한 데이터는 마블 캐릭터, 코믹스, 제작자, 시리즈
- 코드 파일의 크기를 줄이기 위해 github에 올릴 샘플 코드는 마블 캐릭터 수집 부분만 살려둠.

## 1-1) 마블 캐릭터 정보 가져오기
- total : 1,562 EA

In [ ]:
marvel_characters = [] 


# total 캐릭터 1562개를 모두 뽑아내기 위한 반복문
while offset < 1600 : 
    marvel_api = "https://gateway.marvel.com/v1/public/characters?apikey={}&ts=1&hash={}&limit={}&offset={}"
    marvel_api = marvel_api.format(api_public_key, api_hash, limit, offset)
    print("offset: {}".format(offset))

    m_crt_json = requests.get(marvel_api).json()['data']['results']
    print("offset: {}".format(m_crt_json))

    for element in m_crt_json :
        marvel_characters.append(element)

    offset += limit
    
    time.sleep(1)

In [247]:
# 마블 캐릭터 -> json_normalize
df_norm_characters = pd.json_normalize(marvel_characters)
df_norm_characters.head(3)

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17-0400,http://gateway.marvel.com/v1/public/characters...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,http://gateway.marvel.com/v1/public/characters...,...,[{'resourceURI': 'http://gateway.marvel.com/v1...,3,21,http://gateway.marvel.com/v1/public/characters...,[{'resourceURI': 'http://gateway.marvel.com/v1...,20,1,http://gateway.marvel.com/v1/public/characters...,[{'resourceURI': 'http://gateway.marvel.com/v1...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04-0400,http://gateway.marvel.com/v1/public/characters...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,http://gateway.marvel.com/v1/public/characters...,...,[{'resourceURI': 'http://gateway.marvel.com/v1...,2,7,http://gateway.marvel.com/v1/public/characters...,[{'resourceURI': 'http://gateway.marvel.com/v1...,7,0,http://gateway.marvel.com/v1/public/characters...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30-0400,http://gateway.marvel.com/v1/public/characters...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,http://gateway.marvel.com/v1/public/characters...,...,[{'resourceURI': 'http://gateway.marvel.com/v1...,20,57,http://gateway.marvel.com/v1/public/characters...,[{'resourceURI': 'http://gateway.marvel.com/v1...,20,0,http://gateway.marvel.com/v1/public/characters...,[],0


In [ ]:
# 마블 캐릭터에서 필요한 컬럼만 가져와 df 만들고 csv로 저장하기
df_characters = df_norm_characters[["id", "name", "description"]]

df_characters["characters_id"] = df_characters["id"]
df_characters.drop("id", axis=1, inplace=True)

df_characters = df_characters.set_index("characters_id")

df_characters.to_csv("character_ID.csv")
df_characters

## 1-2) 마블 캐릭터 - 코믹스 ID

In [ ]:
# 딕셔너리에 있는 값을 순회하면서 "id"와 "comics"라는 키 값이 존재하는 경우, 
# 그 값을 가져와 "comic_items"라는 새로운 딕셔너리에 추가
# "comics"의 값은 또 다른 딕셔너리이기 때문에, 두 번째 for 루프를 돌며 "items"라는 키 값을 찾음

marvel_characters_comics = []

for character in marvel_characters :

    comic_items = {}

    for k1, v1 in character.items() :
        if k1 == "id":
            comic_items[k1] = v1

        elif k1 == "comics" :

            for k2, v2 in character[k1].items() :
                if k2 == "items" :
                    comic_items[k2] = v2
                    
    marvel_characters_comics.append(comic_items)

marvel_characters_comics[0]


In [366]:
# record_path= : 평탄화 할 위치
# meta= : 포함할 열(column)
df_characters_comics = pd.json_normalize(marvel_characters_comics, record_path="items", meta=["id"])

# apply() 함수는 Pandas DataFrame을 엑셀의 매크로 함수처럼 각 column에 대해 연산을 수행하는 함수
df_characters_comics["comics_id"] = df_characters_comics["resourceURI"].apply(replace_resourceURI_id)
df_characters_comics.drop("resourceURI", axis=1, inplace=True)


df_characters_comics["characters_id"] = df_characters_comics["id"]
df_characters_comics.drop("id", axis=1, inplace=True)


characters_comics = df_characters_comics.set_index("comics_id")

characters_comics.to_csv("characters_comics_ID.csv")
characters_comics

,name,characters_id
comics_id,,
21366,Avengers: The Initiative (2007) #14,1011334
24571,Avengers: The Initiative (2007) #14 (SPOTLIGHT...,1011334
21546,Avengers: The Initiative (2007) #15,1011334
21741,Avengers: The Initiative (2007) #16,1011334
21975,Avengers: The Initiative (2007) #17,1011334
...,...,...
37047,Hulk (2008) #36,1009742
40023,Hulk (2008) #36 (I Am Captain America Variant),1009742
29541,Incredible Hulks (2010) #602 (SHS VARIANT),1009742


## 1-3) 마블 캐릭터 - 시리즈 ID

In [251]:
marvel_characters_series = []

for character in marvel_characters :

    series_items = {}

    for k1, v1 in character.items() :
        if k1 == "id" :
            series_items[k1] = v1

        elif k1 == "series" :

            for k2, v2 in character[k1].items() :
                if k2 == "items" :
                    series_items[k2] = v2
                    
    marvel_characters_series.append(series_items)

marvel_characters_series[0]


{'id': 1011334,
 'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/series/1945',
   'name': 'Avengers: The Initiative (2007 - 2010)'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/series/2005',
   'name': 'Deadpool (1997 - 2002)'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/series/2045',
   'name': 'Marvel Premiere (1972 - 1981)'}]}

In [252]:
# record_path= : 평탄화 할 위치
# meta= : 포함할 열(column)
df_characters_series = pd.json_normalize(marvel_characters_series, record_path="items", meta=["id"])

# apply() 함수는 Pandas DataFrame을 엑셀의 매크로 함수처럼 각 column에 대해 연산을 수행하는 함수
df_characters_series["series_id"] = df_characters_series["resourceURI"].apply(replace_resourceURI_id)
df_characters_series.drop("resourceURI", axis=1, inplace=True)


df_characters_series["characters_id"] = df_characters_series["id"]
df_characters_series.drop("id", axis=1, inplace=True)


characters_series = df_characters_series.set_index("series_id")

characters_series.to_csv("characters_series_ID.csv")
characters_series

,name,characters_id
series_id,,
1945,Avengers: The Initiative (2007 - 2010),1011334
2005,Deadpool (1997 - 2002),1011334
2045,Marvel Premiere (1972 - 1981),1011334
17765,FREE COMIC BOOK DAY 2013 1 (2013),1017100
3374,Hulk (2008 - 2012),1017100
...,...,...
3374,Hulk (2008 - 2012),1011183
3374,Hulk (2008 - 2012),1009742
8842,Incredible Hulks (2010 - 2011),1009742


## 1-4) DB에 넣을 캐릭터-코믹스ID 데이터 만들기
- marvel_characters + marvel_comics_ID

In [368]:
marvel_characters_comics_all = []

for character in marvel_characters :

    all_items = {}

    for k1, v1 in character.items() :
        if k1 == "id" or k1 == "name" or k1 == "description" :
            all_items[k1] = v1
            
        elif k1 == "comics" :
            for k2, v2 in character[k1].items() :
                if k2 == "items" :
                    all_items[k2] = v2

    marvel_characters_comics_all.append(all_items)


marvel_characters_comics_all[0]

{'id': 1011334,
 'name': '3-D Man',
 'description': '',
 'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21366',
   'name': 'Avengers: The Initiative (2007) #14'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/24571',
   'name': 'Avengers: The Initiative (2007) #14 (SPOTLIGHT VARIANT)'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21546',
   'name': 'Avengers: The Initiative (2007) #15'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21741',
   'name': 'Avengers: The Initiative (2007) #16'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/21975',
   'name': 'Avengers: The Initiative (2007) #17'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/22299',
   'name': 'Avengers: The Initiative (2007) #18'},
  {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/22300',
   'name': 'Avengers: The Initiative (2007) #18 (ZOMBIE VARIANT)'},
  {'resourceURI': 'http://gateway.marvel.com/v1/pub

In [348]:
marvel_characters_comics_all_temp = []

for mcc_all in marvel_characters_comics_all :
    
    for item in mcc_all['items'] :

        temp_dict = {}

        temp_dict['characters_id'] = mcc_all['id']
        temp_dict['name'] = mcc_all['name']
        temp_dict['description'] = mcc_all['description']
        temp_dict['comics_id'] = replace_resourceURI_id(item['resourceURI'])
        
        marvel_characters_comics_all_temp.append(temp_dict)
    

# marvel_characters_comics_all_temp[0]
print(len(marvel_characters_comics_all_temp))

17139


In [39]:
# 필요한 컬럼만 가져와 df 만들고 csv로 저장하기
df_series = df_norm_series[["id", "title", "description", "startYear", "endYear", "rating"]]

df_series["series_id"] = df_series["id"]
df_series.drop("id", axis=1, inplace=True)

df_series = df_series.set_index("series_id")

df_series.to_csv("series_ID.csv")
df_series

C:\Users\medici\AppData\Local\Temp\ipykernel_3408\2281071832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_series["series_id"] = df_series["id"]
C:\Users\medici\AppData\Local\Temp\ipykernel_3408\2281071832.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_series.drop("id", axis=1, inplace=True)


,title,description,startYear,endYear,rating
series_id,,,,,
31445,Fantastic Four by Dan Slott Vol. 1 (2021),None,2021,2021,
26024,Superior Spider-Man Vol. 2: Otto-matic (2019),None,2019,2019,
18454,100th Anniversary Special (2014),None,2014,2014,Rated T
13379,15 Love (2011),None,2011,2011,
13380,15-Love GN-TPB (2013 - Present),None,2013,2099,
...,...,...,...,...,...
23085,Zombies Assemble (2017),None,2017,2017,
23388,Zombies Assemble 2 (2017),None,2017,2017,
22383,ZOMBIES ASSEMBLE VOL. 1 MANGA (2017),None,2017,2017,


# **5. mysql DB 생성**
- Python Code로 MySQL DB 

In [49]:
# 파이썬에서 mysql 을 사용하기 위한 모듈 설치
!pip install pymysql

In [48]:
# MySQL 연결 설정
import pymysql.cursors

sql_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='sqlds',
                             charset='utf8mb4')

# marvel_data_DB 데이터베이스(database) 생성
try:
    with sql_connection.cursor() as cursor:
        sql = 'CREATE DATABASE marvel_data_DB'
        cursor.execute(sql)
    sql_connection.commit()
finally:
    sql_connection.close()

In [364]:
# 각 테이블을 넣을 DB를 지정하고 변수에 담음
m_DB_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='sqlds',
                             db="marvel_data_DB",
                             charset='utf8mb4')

# 4개의 테이블 만들기
try:
    with m_DB_connection.cursor() as cursor :

        marvel_characters_sql = '''
            CREATE TABLE marvel_characters (
                idx int auto_increment NOT NULL PRIMARY KEY,
                characters_id int NOT NULL,
                name varchar(100) NOT NULL,
                description TEXT,
                comics_id int NOT NULL
            )
            '''

        marvel_comics_sql = '''
            CREATE TABLE marvel_comics (
                idx int auto_increment NOT NULL PRIMARY KEY,
                comics_id int NOT NULL,
                title varchar(255) NOT NULL,
                description TEXT,
                series_id int NOT NULL
            )
            '''
# foreign key (series_id) references marvel_series(series_id) 를 사용 못한 이유
# OperationalError: (1822, "Failed to add the foreign key constraint. Missing index for constraint 'marvel_comics_ibfk_1' in the referenced table 'marvel_series'")

        marvel_creators_sql = '''
            CREATE TABLE marvel_creators (
                idx int auto_increment NOT NULL PRIMARY KEY,
                creators_id int NOT NULL,
                fullName varchar(255) NOT NULL,
                series_id int NOT NULL
            )
            '''

        marvel_series_sql = '''
            CREATE TABLE marvel_series (
                idx int auto_increment NOT NULL PRIMARY KEY,
                series_id int NOT NULL,
                title varchar(255) NOT NULL,
                description TEXT,
                startYear int NOT NULL,
                endYear int NOT NULL,
                rating varchar(255)
            )
            '''
        
        cursor.execute(marvel_series_sql)
        cursor.execute(marvel_comics_sql)
        cursor.execute(marvel_creators_sql)
        cursor.execute(marvel_characters_sql)

    m_DB_connection.commit()
finally:
    m_DB_connection.close()

In [377]:
m_DB_connection = pymysql.connect(host='localhost',
                             user='root',
                             password='sqlds',
                             db="marvel_data_DB",
                             charset='utf8mb4')

# 테이블에 데이터 insert
try:
    with m_DB_connection.cursor() as cursor:

        characters_sql = 'INSERT INTO marvel_characters (characters_id, name, description, comics_id) VALUES (%s, %s, %s, %s)'
        for m_character in marvel_characters_comics_all_temp : 
            cursor.execute(characters_sql, (m_character["characters_id"], 
                                            m_character["name"], 
                                            m_character["description"],  
                                            m_character["comics_id"]))


        comics_sql = 'INSERT INTO marvel_comics (comics_id, title, description, series_id) VALUES (%s, %s, %s, %s)'
        for m_comic in marvel_comics_series_all : 
            cursor.execute(comics_sql, (m_comic["id"], 
                                        m_comic["name"], 
                                        "", 
                                        m_comic["series_id"]))


        creators_sql = 'INSERT INTO marvel_creators (creators_id, fullName, series_id) VALUES (%s, %s, %s)'
        for m_creator in marvel_creators_series_db : 
            cursor.execute(creators_sql, (m_creator["creators_id"], 
                                          m_creator["fullName"], 
                                          m_creator["series_id"]))


        series_sql = 'INSERT INTO marvel_series (series_id, title, description, startYear, endYear, rating) VALUES (%s, %s, %s, %s, %s, %s)'
        for m_serie in marvel_series : 
            cursor.execute(series_sql, (m_serie["id"], 
                                        m_serie["title"], 
                                        m_serie["description"], 
                                        m_serie["startYear"], 
                                        m_serie["endYear"],
                                        m_serie["rating"]))


    m_DB_connection.commit()
finally:     
    m_DB_connection.close()
